# Extracting test videos from windtunnel:

In [1]:
from windtunnel_py.windtunnel_hdf5 import WindtunnelHDF5
import os
import cv2
import numpy as np
import pandas as pd
import h5py
import tqdm

In [2]:
main_h5_dir = "/Users/marcusschou/Desktop/SDU/data_science/kurser_4/Speciale_Data_Science/INFERENCE_EVAL/h5_files_test"

In [3]:
h5_file_paths = []

for subfolder in os.listdir(main_h5_dir):
    subfolder_path = os.path.join(main_h5_dir, subfolder)
    if os.path.isdir(subfolder_path):
        for h5_file in os.listdir(subfolder_path):
            if not h5_file.startswith("."):
                item_path = os.path.join(subfolder_path, h5_file)
                h5_file_paths.append(item_path)

In [4]:
source_frames = pd.read_csv("/Users/marcusschou/Desktop/SDU/data_science/kurser_4/Speciale_Data_Science/INFERENCE_EVAL/head_annotation_data_nat_updated_v2_sourceVideoFrames.csv")
source_frames.head()

,recDate,sourceVideo,camera,matlabFrameIdx
0,20210805,Windtunnel_Prey_Capture_Approach_1628164847_20...,2,1274
1,20210805,Windtunnel_Prey_Capture_Approach_1628164847_20...,2,1677
2,20210805,Windtunnel_Prey_Capture_Approach_1628164847_20...,2,508
3,20210805,Windtunnel_Prey_Capture_Approach_1628164847_20...,2,1304
4,20210805,Windtunnel_Prey_Capture_Approach_1628164847_20...,2,1235


In [5]:
camera_numbers = [1, 2]
video_dir = "/Users/marcusschou/Desktop/SDU/data_science/kurser_4/Speciale_Data_Science/INFERENCE_EVAL/videos"

def write_video(imgs, output_path="output_video.mp4", fps=15, frame_ids=None):
    height, width, channels, num_frames = imgs.shape
    
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  
    video_writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height), isColor=False)
    
    for i in range(num_frames):
        frame = imgs[:, :, 0, i]  
        frame = np.uint8(frame)   
        video_writer.write(frame)
    
    video_writer.release()

for h5_path in tqdm.tqdm(h5_file_paths):
    h = WindtunnelHDF5(str(h5_path))
    h5_filename = os.path.basename(h5_path)
    frames_in_train = source_frames[source_frames["sourceVideo"] == h5_filename]
    
    if len(frames_in_train) != 0:
        print(f"Number of frames that is in the trainset: {len(frames_in_train)}")
        train_frame_ids = frames_in_train["matlabFrameIdx"].tolist()
        camera_number = frames_in_train["camera"].unique()
        print(f"{train_frame_ids} These indices are used in the training of the model and will be excluded in the test video")
    else:
        camera_number = None
    
    for i in camera_numbers:
        if i == 1:
            imgs, idx = h.camera1.get_frames_uint8(imadjust=True)
        elif i == 2:
            imgs, idx = h.camera2.get_frames_uint8(imadjust=True)
        
        if camera_number is not None and i in camera_number:
            imgs_new = []
            for img, id in zip(imgs, idx):
                if id not in train_frame_ids:
                    imgs_new.append(img)
            if imgs_new:  # Check if imgs_new is not empty
                imgs_new = np.stack(imgs_new, axis=-1)  # Convert to 4D NumPy array
                video_path = os.path.join(video_dir, h5_filename + "_" + f"cam_{i}" + "_" + "CLAHE.mp4")
                write_video(imgs_new, output_path=video_path)
            else:
                print(f"No frames to write for camera {i} in {h5_filename}")
        else:
            video_path = os.path.join(video_dir, h5_filename + "_" + f"cam_{i}" + "_" + "CLAHE.mp4")
            write_video(imgs, output_path=video_path)

  0%|          | 0/9 [00:00<?, ?it/s][ WARN:0@161.095] global cap.cpp:781 open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.11.0) /Users/xperience/GHA-Actions-OpenCV/_work/opencv-python/opencv-python/opencv/modules/videoio/src/cap_images.cpp:267: error: (-215:Assertion failed) number < max_number in function 'icvExtractPattern'


[ WARN:0@196.683] global cap.cpp:781 open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.11.0) /Users/xperience/GHA-Actions-OpenCV/_work/opencv-python/opencv-python/opencv/modules/videoio/src/cap_images.cpp:267: error: (-215:Assertion failed) number < max_number in function 'icvExtractPattern'


 11%|█         | 1/9 [03:08<25:10, 188.82s/it][ WARN:0@215.227] global cap.cpp:781 open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.11.0) /Users/xperience/GHA-Actions-OpenCV/_work/opencv-python/opencv-python/opencv/modules/videoio/src/cap_images.cpp:267: error: (-215:Assertion failed) number < max_number in function 'icvExtractPattern'


[ 